<a href="https://colab.research.google.com/github/Ukbang/Build_with_AI_2025/blob/main/1_DataAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain langgraph langchain-google-genai langchain-core langchain-community langchain-experimental koreanize-matplotlib

In [ ]:
!pip install langchain-openai

In [ ]:
import pandas as pd
import re
import koreanize_matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from typing import Annotated, Literal, Tuple
from typing_extensions import TypedDict
from langchain.schema import AIMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.runnables import RunnableConfig
from langchain_experimental.tools.python.tool import PythonAstREPLTool
from langgraph.prebuilt import ToolNode
from pydantic import BaseModel, Field

from langchain_openai import ChatOpenAI

import base64
import io
import os
import warnings

warnings.filterwarnings("ignore")

In [ ]:
google_api_key = "" # @param {"type":"string"}
openai_api_key = "" # @param {"type":"string"}

In [ ]:
# 환경변수에 Goolge API Key 등록

os.environ["google_api_key"] = google_api_key
os.environ["GOOGLE_API_KEY"] = google_api_key
os.environ["openai_api_key"] = openai_api_key
os.environ["OPENAI_API_KEY"] = openai_api_key

In [ ]:
# 데이터프레임 불러오기

url = "https://raw.githubusercontent.com/Ukbang/Build_with_AI_2025/main/data/titanic.csv"
df = pd.read_csv(url)

In [ ]:
# 데이터프레임 정보 확인

df.info()

In [ ]:
# 그래프 상태 정의

class State(TypedDict):
    messages : Annotated[list, add_messages] # History
    code : Annotated[str, "Python code"]
    code_result : Annotated[str, "Python code Result"]

In [ ]:
# LLM 로드
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash",
                            temperature=0.,
                            convert_system_message_to_human=True,
                            )

# llm =ChatOpenAI(model="gpt-4o-mini",
#            temperature=0.)

# llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash-preview-04-17",
#                             temperature=0.,
#                             convert_system_message_to_human=True,
#                             )

# 파이썬 코드 실행 도구 정의
tool = PythonAstREPLTool(name="python_repl_ast",
                        description="A Python shell. Use this to execute python commands. \
                                    Input should be a valid python command. When using this tool, \
                                    sometimes output is abbreviated - make sure it does not look abbreviated before using it in your answer.",
                         locals={"df":df})

In [ ]:
tool.invoke("```python for i in range(10): print(i)")

In [ ]:
# 데이터셋 제목, 요약 생성 함수

def create_title_summary(df:pd.DataFrame) -> Tuple[str, str]:

    # 데이터셋이 너무 클 시 3000개의 샘플만 제공
    df_sampled = df.sample(n=3000) if len(df) > 3000 else df

    prompt = PromptTemplate.from_template("""
            당신은 요약 전문가입니다.

            데이터셋 : {df}

            데이터셋의 정보를 보고 제목과 요약을 만들어냅니다.
            제목은 이 데이터셋을 가장 잘 표현할 수 있는 제목으로 결정하여야합니다.


            제목:
            요약:

            """
            )

    chain = prompt | llm

    # chain.invoke는 AI Message객체를 반환함. 그 중 str 형태인 content만 추출
    result = chain.invoke({"df":df_sampled})

    title = "Untitled"
    summary = "No Summary"

    # 요약 양식에 맞지 않는 답변을 할 시 "Untitle"과 "No Summary"로 설정하도록 예외처리
    try:
        content = result.content
        content = content.replace("## 결과:\n\n", "")
        lines = content.split("\n")
        title = lines[0].replace("## 제목:", "").strip()
        summary = "\n".join(lines[1:]).replace("## 요약:", "").strip()
    except:
        pass

    print("===== 제목, 요약 생성 완료 =====")

    return title, summary

In [ ]:
# 제목, 요약 생성

title, summary = create_title_summary(df)

In [ ]:
print("제목 : ", title)
print("요약 : ", summary)

In [ ]:
# LLM에게 도구 할당

llm_with_tools = llm.bind_tools([tool])

In [ ]:
llm_with_tools.invoke("1부터 10까지  출력하는 파이썬 코드")

In [ ]:
class HistoryChecker(BaseModel):
    """
    이전의 대화 기록을 참고하여 질문에 대해 답변할 수 있는지 판단합니다.
    답변할 수 있다면 "yes", 답변할 수 없다면 "no"를 반환합니다.
    """

    yes_no : Literal["yes", "no"] = Field(..., description="""Use your previous conversation history to determine if you can answer your questions.
    Return "yes" if you can answer, "no" if you can't answer.""")

In [ ]:
# LLM의 응답을 HistoryChecker 클래스 구조에 맞춰 파싱하도록 설정

history_checker = llm.with_structured_output(HistoryChecker)

In [ ]:
history_checker.invoke("오늘 인천 날씨는 어때?")

In [ ]:
# 히스토리 기반 답변 분기를 위한 함수 설정

def history_check(state:State):

    prompt = PromptTemplate.from_template("""

                이전의 대화 기록을 참고하여 질문에 대해 답변할 수 있는지 판단합니다.
                답변할 수 있다면 "yes", 답변할 수 없다면 "no"를 반환합니다.

                대화 기록 : {history}

                질문 : {query}

                """)

    chain = prompt | history_checker

    result = chain.invoke({"history":state["messages"][:-1],
                            "query":state["messages"][-1]})

    return result.yes_no

In [ ]:
# 식별을 위한 노드

def history_node(state:State):

    return

In [ ]:
# 도구 사용 여부 결정 노드

def select(state:State):
    prompt = PromptTemplate.from_template("""
            당신은 데이터 분석가입니다.
            당신은 데이터프레임인 df를 활용할 수 있습니다.
            당신이 활용 가능한 데이터프레임의 예시는 아래와 같습니다.
            아래의 예시는 `print(df.head())`의 실행결과입니다.
            활용 가능한 데이터프레임을 갖고 있으니 데이터프레임을 새로 생성할 필요 없습니다.
            코드에 한글이 필요하다면 `import koreanize_matplotlib`을 사용하세요.

            당신은 'python_repl_ast'라는 파이썬 코드 실행 도구만을 가지고 있습니다.
            아래의 내용을 참고하여 질문에 대한 코드를 생성합니다.

            df : {df}

            title : {title}
            summary : {summary}

            query : {query}
            """
            )

    chain = prompt | llm_with_tools

    result = chain.invoke({"df":df.head(),
                           "title":title,
                            "summary":summary,
                            "query":state["messages"][-1].content})

    if hasattr(result, "tool_calls") and len(result.tool_calls) > 0:
        tool_calls = result.tool_calls

        # 코드 추출
        code = tool_calls[0]["args"]["query"]
        return {"code": code}
    else:
        # 도구 사용을 안한다면 코드는 공백으로 설정
        return {"code":""}

In [ ]:
# 코드 실행 노드

def code_executor(state:State):

    try:
        # 코드 실행 결과인 이미지 추출
        if "plt" in state["code"] or "sns" in state["code"]:
            save_fig = """
import io
import base64

buf = io.BytesIO()
plt.savefig(buf, format="png")
buf.seek(0)

encoded = base64.b64encode(buf.read()).decode("utf-8")

print(encoded)
"""

            execute_code = state["code"] + save_fig

            result = tool.invoke(execute_code)

            return {"code_result":result}
        # 시각화가 아닌 경우 코드만 실행
        else:
            result = tool.invoke(state["code"])
            return {"code_result":result}
    except:
        return {"code_result": ""}

In [ ]:
# 코드에 의한 답변 노드

def code_response(state:State):
    prompt = ChatPromptTemplate.from_messages([
        ("system", """코드 : {code} \n
                    결과 : {code_result}
        당신은 주어진 코드와 코드 결과를 바탕으로 질의에 대해 답변합니다.
        절대 코드에 대해 설명하지마세요.
        독자는 프로그래머가 아닙니다.
        항상 출력되는 값을 기준으로 설명합니다.
        숫자가 매우 중요합니다. 숫자에 대한 정보를 잊지 마세요.
        데이터 분석과 관련된 코드가 입력된다면 항상 인사이트를 포함하세요. """),
        ("human", '{query}')
    ])

    chain = prompt | llm

    result = chain.invoke({"code":state["code"],
                        "code_result":state["code_result"],
                        "query":state["messages"][-1]})

    return { "messages": result}

In [ ]:
# 기억 및 일반 응답 노드

def response(state:State):
    prompt = PromptTemplate.from_template("""

                이전의 대화 기록을 참고하여 질문에 대해 답변하세요.
                아래 대화 기록을 첨부합니다.
                대화 기록을 통해 답변이 어렵다면 내부 지식을 참조하세요.

                대화 기록 : {history}

                질문 : {query}

                """)

    chain = prompt | llm

    result = chain.invoke({"history":state["messages"][:-1],
                           "query":state["messages"][-1]})

    return {"answer":result.content,
            "messages":result}

In [ ]:
# 그래프 정의

graph_builder = StateGraph(State)

In [ ]:
# 노드 및 엣지 정의

graph_builder.add_node("history_node", history_node)
graph_builder.add_node("select", select)
graph_builder.add_node("code_executor", code_executor)
graph_builder.add_node("code_response", code_response)
graph_builder.add_node("response", response)


graph_builder.add_edge(START, "history_node")
graph_builder.add_conditional_edges("history_node",
                                    history_check,
                                    {
                                    "no":"select",
                                     "yes":"response"
                                     }
                                     )
graph_builder.add_edge("select", "code_executor")
graph_builder.add_edge("code_executor", "code_response")
graph_builder.add_edge("code_response", END)
graph_builder.add_edge("response", END);

In [ ]:
# 메모리 설정 및 그래프 컴파일

memory = MemorySaver()

graph = graph_builder.compile(checkpointer=memory)

In [ ]:
# 그래프 시각화
# 가끔 "ReadTimeout: HTTPSConnectionPool(host='mermaid.ink', port=443): Read timed out. (read timeout=10)"라는 에러가 발생
# 시간 초과로 그래프 생성에 실패했다는 메시지일뿐 기능과는 관계없으니 진행해도 괜찮습니다.

graph

In [ ]:
# 사용할 설정값 정의

config = RunnableConfig(recursion_limit=10, configurable={"thread_id": "12"})

In [ ]:
# 출력 함수 정의
# mode = "values" : 상태의 키, 값의 형태로 반환
# mode = "updates" : 업데이트되는 값만 반환

def streaming(query, config, mode="values"):

    result = graph.stream({"messages":("user", query)}, config=config, stream_mode=mode)

    if mode == "values":
        for step in result:
            for k, v in step.items():
                if k == "messages":
                    v[-1].pretty_print()
                    print("\n\n")
    elif mode == "updates":
        for step in result:
            for k,v in step.items():
                print(f"\n\n=== {k} ===\n\n")
                print(v)

    return

In [ ]:
streaming("생존자 비율 시각화하고 인사이트 제공해줘", config)

In [ ]:
streaming("아까 내가 질문했던 내용 다시 알려줘", config)

In [ ]:
streaming("아까 물어봤던 숫자들 다 더하면 몇인지 알려줘", config)

In [ ]:
# 올림픽 데이터 로드 및 도구 재정의

url = "https://raw.githubusercontent.com/Ukbang/Build_with_AI_2025/main/data/athlete_events.csv"
df = pd.read_csv(url)

tool = PythonAstREPLTool(name="python_repl_ast",
                        description="A Python shell. Use this to execute python commands. \
                                    Input should be a valid python command. When using this tool, \
                                    sometimes output is abbreviated - make sure it does not look abbreviated before using it in your answer.",
                        locals={"df":df})


# 제목, 요약 생성
title, summary = create_title_summary(df)

In [ ]:
print("제목 : ", title)
print()
print("요약 : ", summary)

In [ ]:
config = RunnableConfig(recursion_limit=10, configurable={"thread_id": "99"})

streaming("올림픽에는 몇개의 나라가 출전했나요?", config)

In [ ]:
streaming("가장 많이 출전한 나라는 어디인가요?", config)

In [ ]:
df["NOC"].value_counts()

In [ ]:
streaming("올림픽에 출전한 선수들의 평균 체중은 얼마인가요?", config)

In [ ]:
df["Weight"].mean()

In [ ]:
config = RunnableConfig(recursion_limit=10, configurable={"thread_id": "999"})

streaming("키와 체중, 그리고 메달 획득과의 상관관계를 보여주세요. 산점도 그래프로 그려주세요.", config)